In [ ]:
from __future__ import annotations

from typing import Any
import cProfile
import pstats
from asyncio import sleep

In [ ]:
from py_research.db import DataSource, RelPush, RelPull, DataBase, RecMap, All, SelIdx

In [5]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [ ]:
async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [ ]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelPush(
            target=Search.results,
            edge_map=RecMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelPush(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelPush(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelPush(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    edge_map=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: RelPull(
                    sel=All,
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIdx()
            },
        ),
    },
)

In [8]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [ ]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.to_db([data], db=db)

    stats = pstats.Stats(pr)

In [ ]:
import polars as pl

pl.read_database("SELECT * FROM schema_Organization_countries", db.engine)

_from__id,idx,value
str,i64,str
"""294904d0-e719-4599-899a-30bb20…",0,"""USA"""
"""294904d0-e719-4599-899a-30bb20…",1,"""Mexico"""
"""294904d0-e719-4599-899a-30bb20…",2,"""Canada"""
"""65d054da-b302-4728-aac0-8a1698…",0,"""USA"""
"""65d054da-b302-4728-aac0-8a1698…",1,"""Canada"""
"""5f96952f-62aa-46da-b2e4-842a90…",0,"""USA"""
"""5f96952f-62aa-46da-b2e4-842a90…",1,"""Mexico"""


In [ ]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db2 = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.to_db([data], db=db2)

    stats = pstats.Stats(pr)

Post-loading: `Assignment`: 100%|██████████| 6/6 [00:00<00:00, 159.93it/s]


In [ ]:
user_key_0 = db[User].keys()[0]
user_key_0

'1f90a58f-a520-4527-9c52-07f25dd33bad'

In [ ]:
del db[User][user_key_0]

In [ ]:
db[User].load()

_id,name,age
str,str,i64
"""4f4f0578-e7d4-4d92-882c-73f02d…","""John""",31
"""dc671c32-892c-4008-af31-9ada14…","""Jane""",56


In [ ]:
db2[User].load()

_id,name,age
str,str,i64
"""16c6b407-45ff-4d35-b12b-6cf1d0…","""John""",31
"""7b540138-97c2-43ac-ab0d-578171…","""Jane""",56
"""8b21ee7b-85ec-41db-91a7-bc86b4…","""Jack""",66


In [15]:
db3 = db | db2
db3.describe()


{
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 6,
            'schema.Task': 12,
            'schema.User': 5,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 12, 'schema.Membership': 14}
    },
    'backend': None,
    'overlay': 'upsert/(a7d8c35b1d|0813e81f13)/c9ba0523'
}

In [16]:
db4 = db << db2
db4.describe()


{
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 6,
            'schema.Task': 12,
            'schema.User': 5,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 14},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 12, 'schema.Membership': 14}
    },
    'backend': None,
    'overlay': 'insert/(a7d8c35b1d<<0813e81f13)/a244af2f'
}

In [ ]:
db5 = db ^ db2
db5.describe()


{
    'contents': {
        'records': {
            'schema.Project': 3,
            'schema.Organization': 3,
            'schema.Task': 6,
            'schema.User': 2,
            'schema.Search': 1
        },
        'arrays': {'schema.Organization.countries': 7},
        'relations': {'schema.SearchResult': 3, 'schema.Assignment': 6, 'schema.Membership': 7}
    },
    'backend': None,
    'overlay': 'update/(a7d8c35b1d>>0813e81f13)/6d2fe804'
}